# Bekins BHSA to NLTK Trees 1.0

## Overview 

This notebook experiments with building syntax trees from BHSA data using the NLTK Tree object. The basic approach is: 

1) Build a base tree from the distributional nodes (sentence_atom, clause_atom, and phrase_atom)
2) Move distributional nodes under the proper functional node (sentence, clause, and phrase)
3) Move daughter nodes under the mother where a dependent relationship is specified
4) Move daughter nodes under a common node with the mother where a parallel or coordinated relationship is specified
5) Clean up the tree by trimming empty nodes and collapsing redundant nodes

I took inspiration from:

https://github.com/ETCBC/trees/blob/master/programs/trees.ipynb  
https://github.com/ch-jensen/BHSA2RRG/blob/main/BHSA2RRG.ipynb

Note that I have taken a slightly different approach by converting to a NLTK Tree first and then rearranging. Note that this is not a particularly elegant or efficient way to do it, but it seems to work OK. More importantly, it will now allow me to use other tools in the NLTK package for grammatical analysis. 

In [ ]:
# Imports
from nltk import *
from tf.app import use
import random
A = use('bhsa', hoist=globals())

### 1. Helper functions

The following functions are used to get the tree position of a node, the node at a certain tree position, or the label of the node at that position.

In [ ]:
def get_pos(node, tree):
    ''' given the number of a node and a tree, returns the pos of that node in the tree '''
    for pos in tree.treepositions():
        try:
            tree[pos].label()
        except AttributeError:
            pass
        else:
            #pos_node = int(tree[pos].label()[tree[pos].label().index(':')+1:])
            pos_node = int(get_node(pos, tree))
            if node == pos_node:
                node_pos = pos
                
    return node_pos

def get_node(pos, tree):
    '''given a position in a tree, returns the number of the node at that position'''
    try:
        node = tree[pos].label()[tree[pos].label().index(':')+1:]
    except:
        node = None
        
    return node

def get_label(pos,tree):
    '''given a position in a tree, returns the label of the tree at that position'''
    try:
        label = tree[pos].label()[:tree[pos].label().index(':')]
    except:
        label = tree[pos].label()
        
    return label
        

### 2. Tree_string

This function builds a base tree string from the BHSA distributional objects (\_atom) by using the oslots to ascertain embeddedness. This works because the distributional nodes are never discontinuous.

In [ ]:
def tree_string(node):
    
    ''' Takes BHSA node as root and adds all lower distributional objects to a tree '''
    
    slots = []
    slotlist = []
    for s in E.oslots.s(node):
        slotlist.append(s)
    slots.append((node, slotlist))
    
    for n in L.d(node):
        if F.otype.v(n) == 'word':
            slots.append((n, [E.oslots.s(n)[0]]))
        else:
            slotlist = []
            for s in E.oslots.s(n):
                slotlist.append(s)
            slots.append((n, slotlist))
    parse_string = []
    root_node = '(S:' + str(node)
    parse_string.append(root_node)

    for slot in slots[0][1]:
        parse_string.append(slot)
    
    for (label, slot) in slots:
    
        l = ''
        add = False
        if F.otype.v(label) == 'sentence_atom':
            l = 'S_A'
            add = True
        elif F.otype.v(label) == 'clause_atom':
            l = 'C_A'
            add = True
        elif F.otype.v(label) == 'phrase_atom':
            l = 'P_A'
            add = True
        elif F.otype.v(label) == 'subphrase':
            func = F.function.v(label)
            if func == None:
                l = 'U'
            else:
                l = func
            add = True

        elif F.otype.v(label) == 'word':
            l = F.sp.v(label)
            add = True
        
        l = l + ":" + str(label)
    
        if add:
            open_index = parse_string.index(slot[0])
            parse_string.insert(open_index,'('+l)
            close_index = parse_string.index(slot[-1])
            parse_string.insert(close_index + 1,')')

    parse_string.append(')')
    new_string = []
    for n in parse_string:
        if type(n) == int:
            #new_string.append(F.g_cons.v(n))
            new_string.append(str(n))
        else:
            new_string.append(n)
        
    return(' '.join(new_string))

### 3. Move_func

This function combines the distributional nodes under the appropriate functional nodes. At this stage, discontinuity may be introduced.

In [ ]:
def move_func(t):
        
    ''' Takes distributional tree and rearranges under functional nodes '''
    levels = [('phrase', 'P'), ('clause','C')]
    for level in levels:
        atoms = []
        for pos in t.treepositions():
            try:
                t[pos].label()
            except AttributeError:
                pass
            else:
                label = t[pos].label()[:t[pos].label().index(':')]
                if label == level[1] +'_A':
                    atoms.append((pos, label, t[pos].leaves()))

        units = []
        labels = []
        for n in L.d(node):
            if F.otype.v(n) == level[0]:
                label = level[1] + ':' + str(n)
                labels.append(label)
                units.append((label, E.oslots.s(n)))
        units_table = {k:[] for k in labels}

        # for each unit find all atoms contained in that unit and append tree pos to dict
        for unit in units:
            for atom in atoms:
                result =  all(int(elem) in unit[1] for elem in atom[2])
                if result:
                    units_table[unit[0]].append(atom[0])

        # iterate through labels and build tree from atoms 
        new_trees = []
        deletes = []
        for label in labels:
            unit_atoms = units_table[label]
            trees = []
            i = unit_atoms[0][:-1]
            for atom in unit_atoms:
                tree = t[atom]
                trees.append(tree)
            new_tree = Tree(label, trees)
            new_trees.append((i, new_tree))
            for atom in unit_atoms:
                deletes.append(atom)
                
        # delete subtrees that are going to move
        for pos in reversed(t.treepositions()):
            if pos in deletes:
                del t[pos]

        # add subtrees in new spot
        for tree in reversed(new_trees):
            t[tree[0]].insert(0, tree[1])

### 4. Move_daughter

This function finds nodes with dependent relationships to a mother node and moves the daughter under the mother

In [ ]:
def move_daughter(t, relations):
    
    ''' Given a functional tree, rearranges dependent phrases into a hierarchical structure. '''

    phrases = []
    
    # traverse tree to find all nodes that may have dependent relations
    for pos in reversed(t.treepositions()):
        try:
            t[pos].label()
        except AttributeError:
            pass
        else:
            label = get_label(pos, t)
            if label == 'U' or label == 'P_A' or label == 'C':
                node = int(get_node(pos, t))
                rela = F.rela.v(node)
                if rela in relations:
                    mother = E.mother.f(node)[0]
                    phrases.append((node, mother, rela))

    for phrase in phrases:
        m_node = phrase[1]
        m_pos = get_pos(m_node, t)
        d_node = phrase[0]
        d_pos = get_pos(d_node, t)
        copy_tree = t[d_pos]
        
        mother_pos = t[m_pos].leaves()
        daughter_pos = copy_tree.leaves()

        if F.otype.v(m_node) == 'word' or phrase[2] == 'Link':
            # in these cases insert daughter into node *above* mother
            # insert point will be based on position of mother node in this node
            i_tree = m_pos[:-1]
            if mother_pos[0] < daughter_pos[0]:
                i_point = m_pos[-1:][0] + 1
            else:
                i_point = m_pos[-1:][0] - 1
        else:
            # otherwise insert point is position of daughter relative to mother
            i_tree = m_pos
            if mother_pos[0] < daughter_pos[0]:
                try:
                    i_point = mother_pos.index(str(int(daughter_pos[0])-1)) + 1
                except:
                    i_point = len(mother_pos)
            else:
                i_point = 0
        
        ## move tree by inserting at new position and then deleting at old position
        t[i_tree].insert(i_point, copy_tree)
        del t[d_pos]
        

### 5. Move_para

This function takes nodes with parallel relationships to a mother node and moves the parallel mother and daughter to the same level within a new node 

In [ ]:
def move_para(t):
    
    ''' Given a tree, rearranges parrallel phrases under a new common node. '''
    relations = ['Para', 'Coor']
    phrases = []
    
    # find phrases with relation 'Para' or 'Coor'
    for pos in t.treepositions():
        try:
            t[pos].label()
        except AttributeError:
            pass
        else:
            label = get_label(pos, t)
            if label == 'P' or label == 'P_A' or label == 'C':
                node = int(get_node(pos, t))
                rela = F.rela.v(node)
                if rela in relations:
                    mother = E.mother.f(node)[0]
                    phrases.append((node, mother, rela))

    
    mothers = []
    daughters = []

    for phrase in phrases:
        mother = phrase[1]
        daughter = phrase[0]
        mothers.append(mother)
        daughters.append((mother, daughter))
    
    # build a dict since there may be more than one daughter parallel to the same mother
    table = {k:[] for k in mothers}
    for d in daughters:
        table[d[0]].append(d[1])
        

    ## get pos for mother and all daughters then build new tree under common node
    for mother in reversed(table):
        deletes = []
        m_pos = get_pos(mother, t)
        para_trees = []
        para_trees.append(t[m_pos])
        for daughter in table[mother]:
            d_pos = get_pos(daughter, t)
            para_trees.append(t[d_pos])
            del t[d_pos] # delete daughter at current position
        old_label = get_label(m_pos, t)
        new_label = old_label + ':000'
        new_tree = Tree(new_label, para_trees)
        
        # delete mother at old position
        del t[m_pos]

        # insert new para phrase in mother position
        i_tree = m_pos[:-1]
        i_point = m_pos[-1:][0]
        t[i_tree].insert(i_point, new_tree)


### 6. Cleanup

These functions clean up the resulting tree by trimming empty nodes, collapsing redundant nodes, and relabelling the nodes according to phrase type, function, etc.

In [ ]:
def trim_nodes(t):
    ''' This function finds empty nodes left after rearrangement and trims them off. '''
    deletes = []
    for pos in t.treepositions():
        if len(t[pos]) == 0:
            deletes.append(pos)
    
    for delete in reversed(deletes):
        del t[delete]

In [ ]:
def collapse_nodes(t):
    '''this function looks for redundant nodes and collapses them'''
    adds = []
    for pos in reversed(t.treepositions()[1:]):
        try:
            t[pos].label()
        except AttributeError:
            pass
        else:
            match = all(elem in t[pos].leaves() for elem in t[pos[:-1]].leaves())
            if ((match and ((t[pos].label()[0] == t[pos[:-1]].label()[0]) or (t[pos].label()[0] == 'U' and t[pos[:-1]].label()[0] == 'P'))) 
                or (t[pos].label().startswith('C_A'))):
                children = t[pos][0:]
                i_point = pos[-1:][0]
                del t[pos]
                for child in reversed(children):
                    t[pos[:-1]].insert(i_point, child)



In [ ]:
def relabel_tree(t):
    ''' This function changes tree labels to reflect phrase/clause type and function as necessary'''
    
    # these dicts are for converting labels to my preferred (often simpler) form, but this part can use some work 
    label_dict = {'adjv': 'adj',
                  'adjective': 'adj',
                  'advb': 'adv',
                  'adverb': 'adv',
                  'art': 'def',
                  'article': 'def',
                  'conj': 'conj',
                  'conjunction': 'conj',
                  'inrg': 'ir',
                  'interrogative': 'ir',
                  'intj': 'ij',
                  'interjection': 'ij',
                  'nega': 'neg',
                  'negative': 'neg',
                  'nmpr': 'pn',
                  'pronoun': 'pro',
                  'prde': 'pro-dem',
                  'prep': 'p',
                  'preposition': 'p',
                  'prin': 'pro-int',
                  'prps': 'pro-ps',
                  'subs': 'n',
                  'noun': 'n',
                  'verb': 'v',
                 }
    np = ['NP', 'PrNP', 'PPrP', 'DPrP', 'IPrP'] 
    
    sub_dict = {'adv':'AdvP',
                'n':'NP',
                'pn':'NP',
                'def':'NP',  # need to double check this
                'adj':'AdjP',
                'p':'PP',
                'v':'VP' # this is going to hit on some infinitives/participles, not sure if really VP
               }
    
    # traverse the tree and update each label
    for pos in reversed(t.treepositions()):
        try:
            t[pos].label()
        except AttributeError:
            pass
        else:
            label = get_label(pos, t)
            try:
                t[pos].set_label(label_dict[label])
            except:  
                node = int(get_node(pos,t))
                node_type = F.otype.v(node)
                func = F.function.v(node)
                rela = F.rela.v(node)
                typ = F.typ.v(node)
                if typ in np:
                    typ = 'NP'
                
                if node == 0:
                    new_label = t[pos][0].label()
                elif node_type == 'sentence' or node_type == 'sentence_atom':
                    new_label = 'SENT'
                elif node_type == 'clause' or node_type == 'clause_atom':
                    if rela == 'NA':
                        new_label = 'CLAUSE'
                    else:
                        new_label = 'CLAUSE_' + rela
                else:
                    if func == None:
                        if rela == 'NA' or rela == None:
                            func = ''
                        else:
                            func = '_' + rela
                    else:
                        func = '_' + func
                    
                    if typ == None:
                        ## need function to get type of subphrase
                        if node_type == 'subphrase':
                            try:
                                lower = t[pos][0].label()
                                if lower == 'def':
                                    lower = t[pos][1].label()
                                typ = sub_dict[lower]
                            except:
                                typ = 'SP'
                        else:
                            typ = node_type
                    new_label = typ + func
                t[pos].set_label(new_label)
    

### 6. Build tree

This is the main function that applies the steps in building the tree

In [ ]:
def build_tree(node):
    
    ''' Given a BHSA node, calls tree_string to build a tree from all lower distributional nodes then rearranges and cleans up. '''
    # 1. Build tree using atoms
    t = Tree.fromstring(tree_string(node))
    
    # 2. Move distributional nodes under function
    move_func(t)
    
    # 3. Deal with mother/daughter relationships
    relations = ['rec','atr','mod','adj','dem','Appo', 'Spec', 'Attr', 'Objc','Adju', 'Cmpl', 'RgRc', 'ReVo']
    move_daughter(t, relations)
    
    # 4. Deal with parallel and coordinated relations
    move_para(t)   
    
    # 5. Move linking conjunction under new para phrase
    relations = ['Link']
    move_daughter(t, relations) 
            
    # 5. Trim off empty nodes
    trim_nodes(t)
    
    # 6. Collapse redundant nodes
    collapse_nodes(t)
    
    # 7. Relabel nodes with phrase functions
    relabel_tree(t)
    
    return(t)


### 7. Print tree
These functions help for printing the discontinuous trees.

In [ ]:
def add_words(t):
    ''' This replaces the numbers at the bottom of the tree with a phonological representation of the word for display'''
    for pos in t.treepositions():
        try:
            t[pos].label()
        except:
            t[pos] = F.phono.v(int(t[pos]))


In [ ]:
def reference(node):
    book = L.u(node, 'book')
    chap = L.u(node, 'chapter')
    verse = L.i(node, 'verse')
    bk = F.book.v(book[0])
    ch = F.chapter.v(chap[0])
    if len(verse) > 1:
        vs_s = F.verse.v(verse[0])
        vs_e = F.verse.v(verse[-1])
        print('{} {}:{}-{}'.format(bk, ch, vs_s, vs_e))
    else:
        vs = F.verse.v(verse[0])
        print('{} {}:{}'.format(bk, ch, vs))

In [ ]:
def print_tree(t, node):
    slots = E.oslots.s(node)
    words = []
    for slot in slots:
        words.append(F.phono.v(slot))
                     
    for pos in t.treepositions():
        try:
            t[pos].label()
        except:
            t[pos] = slots.index(int(t[pos]))
            
    t.pretty_print(sentence=words)


### 8. Representative Examples

The following examples demonstrate various interesting structures.

In [ ]:
sentences = [1182832, 1203152, 1202266, 1177385, 1195466, 1201130, 1187920, 1190908, 1203980, 1179845, 1187433, 1226051]

In [ ]:
for node in sentences:
    print(node)
    reference(node)
    t = build_tree(node)
    print_tree(t, node)

### 10. Next Steps

At this point I am still building trees on the fly in order to test the functions. There are a number of constructions that are not yet handled the way I would like. Once I have refined the output to my satisfaction, I will export the whole Hebrew Bible in tree from for analysis.  Among the areas I would like to look at next is parsing the lower level phrase structures, particularly PPs, where the BHSA has left things flat. 